In [1]:
from ModelsContainer import ModelsContainer
from energyflow.datasets import qg_jets
from energyflow.utils import data_split
from sklearn.metrics import roc_auc_score
import scipy.special
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import sys
import os

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("GPUs Available: ", tf.config.list_physical_devices('GPU'))




In [ ]:
# Directory Handling


# get the current working directory
current_working_directory = os.getcwd()

# print output to the console
print(current_working_directory)